In [1]:
import xml, os, pandas as pd, glob
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import fromstring, ElementTree
import requests, re
import time
import requests
import string
from bs4 import BeautifulSoup
from tqdm import tqdm
import random

In [2]:
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data")
npp_ts = pd.read_csv('npp_timespans.csv', sep = ';')

In [3]:
os.chdir("D://Scriptie//Data//csv-metadata")
list_csv = glob.glob('*.csv')

In [4]:
def GetTS(newspaper_title):
    sy = npp_ts.loc[npp_ts['npp'] == newspaper_title, 'sy'].iloc[0]
    ey = npp_ts.loc[npp_ts['npp'] == newspaper_title, 'ey'].iloc[0]
    t = ey - sy
    return t

In [5]:
def clean_and_split_str(txt):
    #strip_special_chars = re.compile("[^A-Za-z0-9#]+")
    translator = str.maketrans('', '', string.punctuation)
    txt = txt.translate(translator)
    txt = re.sub('\s+', ' ', txt).strip()
    txt = txt.lower()
    return txt

In [6]:
url1= "http://services.kb.nl/mdo/oai?verb=GetRecord&identifier=DDD:"
url3 = "&metadataPrefix=didl"

In [7]:
def DateXML(soup_element):
    t = str(soup_element.find_all("date")[0])[6:10]
    return t

In [27]:
def OCRXML(soup_element):
    t = soup_element.find_all("OCRConfidencelevel")
    t = [str(i) for i in t]
    t = [i.replace('<OCRConfidencelevel>', '') for i in t]
    t = [i.replace('<OCRConfidencelevel/>', '') for i in t]
    t = [i.replace('</OCRConfidencelevel>', '') for i in t]
    t = [i for i in t if i]
    t = [int(i.replace('0.', '')) for i in t]
    if len(t) > 0:
        t = (sum(t) / len(t)) / 1000
    return t

In [19]:
def GetTitleNppYear(newspaper_title, year):
    fn = [fn for fn in list_csv if fn[0:4] == newspaper_title][0]
    
    df = pd.read_csv(fn, sep = '\t')
    df['id'] = df.id.str[:-6]
    df['date'] = df.date.str.slice(0,4)
    df= df[df['id'].str[0:3] == 'ddd']
    
    df = df.drop_duplicates(subset='id', keep="first")
    df = df[df['date'] == str(year)]
    if len(df) > 0:    

        df = df.sample(n=30)
        df = df.reset_index(drop=True)

        titles_list = list()

        for issue in list(df.id):
            url = url1 + issue + url3
            result = requests.get(url)
            content = result.content
            soup = BeautifulSoup(content, "xml")
            list_titles = soup.find_all("title")
            list_titles = [str(i) for i in list_titles]
            list_titles = [i.replace('<title>', '') for i in list_titles]
            list_titles = [i.replace('</title>', '') for i in list_titles]
            list_titles = [clean_and_split_str(i) for i in list_titles]

            for title in list_titles:
                t = [title, OCRXML(soup), issue]
                titles_list.append(t)
    if len(df) == 0:
        titles_list = [[year, 'no titles', 'in issue']]
    
    return titles_list
        
    

In [28]:
for npp in tqdm(list(npp_ts.npp)):
    npp_df = pd.DataFrame()

    for year in list(range(int(npp_ts[npp_ts.npp == npp].sy), int(npp_ts[npp_ts.npp == npp].ey))):
        dl = GetTitleNppYear('nesc', year)
        dl = pd.DataFrame(dl, columns = ['title', 'ocr', 'issue'])
        dl['year'] = year
        npp_df = npp_df.append(dl)
    
    fn = npp + "_titles.csv"
    npp_df.to_csv(fn)


 52%|████████████████████████████████████████████████████████████████████████████████████████                                                                                | 11/21 [1:20:15<1:27:39, 525.97s/it]

ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [26]:
GetTitleNppYear('leco', 1868)

[['leeuwarder courant', 0.79175, 'ddd:010585694:mpeg21'],
 ['advertentie', 0.79175, 'ddd:010585694:mpeg21'],
 ['weenen 28 februarij', 0.79175, 'ddd:010585694:mpeg21'],
 ['parijs 27 februarij', 0.79175, 'ddd:010585694:mpeg21'],
 ['londen 28 februarij', 0.79175, 'ddd:010585694:mpeg21'],
 ['berlijn 26 februarij', 0.79175, 'ddd:010585694:mpeg21'],
 ['s gravenhage 28 januarij', 0.79175, 'ddd:010585694:mpeg21'],
 ['leeuwarden 2 maart', 0.79175, 'ddd:010585694:mpeg21'],
 ['leeuwarden 2 maart', 0.79175, 'ddd:010585694:mpeg21'],
 ['familiebericht', 0.79175, 'ddd:010585694:mpeg21'],
 ['familiebericht', 0.79175, 'ddd:010585694:mpeg21'],
 ['weerkundige waarnemingen te leeuwarden', 0.79175, 'ddd:010585694:mpeg21'],
 ['letternieuws', 0.79175, 'ddd:010585694:mpeg21'],
 ['per telegraaf parijs 1 maart', 0.79175, 'ddd:010585694:mpeg21'],
 ['s gravenhage 2 maart', 0.79175, 'ddd:010585694:mpeg21'],
 ['amsterdam 2 maart', 0.79175, 'ddd:010585694:mpeg21'],
 ['advertentie', 0.79175, 'ddd:010585694:mpeg21'],


In [63]:
for year in list(set(npp_df.year)):
    tmp = npp_df[npp_df.year == year]
    
    issue_with_b = 0
    for issue in list(set(npp_df.issue)):
        tmpiss = tmp[tmp.issue == issue]
        
        tt = list(tmpiss.title)
        
        if any("buitenland" in s for s in tt):
            issue_with_b += 1
    print(str(year) + " | " + str(issue_with_b / len(list(set(tmp.issue))) * 100))

1815 | 90.0
1816 | 96.66666666666667
1817 | 100.0
1818 | 100.0
1819 | 100.0
1820 | 100.0
1821 | 100.0
1822 | 90.0
1823 | 100.0
1824 | 93.33333333333333
1825 | 100.0
1826 | 93.33333333333333
1827 | 80.0
1828 | 93.33333333333333
1829 | 100.0
1830 | 90.0
1831 | 96.66666666666667
1832 | 96.66666666666667
1833 | 93.33333333333333
1834 | 96.66666666666667
1835 | 100.0
1836 | 100.0
1837 | 93.33333333333333
1838 | 96.66666666666667
1839 | 100.0
1840 | 100.0
1841 | 96.66666666666667
1842 | 96.66666666666667
1843 | 90.0
1844 | 90.0
1845 | 100.0
1846 | 100.0
1847 | 83.33333333333334
1848 | 83.33333333333334
1849 | 96.66666666666667
1850 | 96.66666666666667
1851 | 96.66666666666667
1852 | 100.0
1853 | 86.66666666666667
1854 | 76.66666666666667
1855 | 100.0
1856 | 96.66666666666667
1857 | 100.0
1858 | 100.0
1859 | 93.33333333333333
1860 | 100.0
1861 | 100.0
1862 | 90.0
1863 | 100.0
1864 | 96.66666666666667
1865 | 100.0
1866 | 96.66666666666667
1867 | 96.66666666666667
1868 | 100.0
1869 | 96.6666666

In [45]:
tmp = npp_df[npp_df.year == 1815]


In [47]:
tmp = tmp[tmp.issue == 'ddd:000023100:mpeg21']

In [53]:
tt= list(tmp.title)

In [57]:
if any("buitenland" in s for s in tt):
    print('yes')

yes
